<a href="https://colab.research.google.com/github/disha-cpu/Resources-For-Stat-Data/blob/master/Feature-wise-dummy-code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [685]:
# importing libraries
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
# from plotnine import *
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
import plotly.express as px
import plotly.graph_objects as go
import random
import math
import statistics
import collections
# from plotnine import ggplot, aes, geom_line
%matplotlib inline

In [686]:
# reading data
data = pd.read_csv('/content/weatherGOA.csv')
# formatting time
data = data.replace(to_replace ="24:00:00", value ="00:00")
data['time'] = data['Date'] +" "+ data['UT time'] 
data['time'] = pd.to_datetime(data['time'])
# converting temprature from K to C
data = data[data['time'].dt.minute == 0]
data['Temperature (C)'] = data['Temperature (K)'] - 273.15
data['Hour'] = data['time'].dt.hour.astype(float)
data['TIME'] = pd.to_numeric(pd.to_datetime(data['time'])).astype(float)

In [687]:
winter = data[data.time.dt.month.isin([1,2,11,12])]
winter = data[data.time.dt.year.isin([2017,2018,2019])]
winter = winter.sort_values(by=['time'])
summer = data[data.time.dt.month.isin([3,4,5,6])]
summer = data[data.time.dt.year.isin([2017,2018,2019])]
summer = summer.sort_values(by=['time'])
monsoon = data[data.time.dt.month.isin([7,8,9,10])]
monsoon = data[data.time.dt.year.isin([2017,2018,2019])]
monsoon = monsoon.sort_values(by=['time'])
winter_dataset = winter[["TIME",'time','Hour',"Temperature (C)","Relative Humidity (%)", "Pressure (hPa)"]]
summer_dataset = summer[['TIME','time','Hour',"Temperature (C)","Relative Humidity (%)", "Pressure (hPa)"]]
monsoon_dataset = monsoon[['TIME','time','Hour',"Temperature (C)","Relative Humidity (%)", "Pressure (hPa)"]]


In [688]:
print(f"mean-temperature: {statistics.mean(winter_dataset['Temperature (C)'])}")
print(f"mean-humidity: {statistics.mean(winter_dataset['Relative Humidity (%)'])}")
print(f"mean-pressure: {statistics.mean(winter_dataset['Pressure (hPa)'])}")
print(f"Temp-count{collections.Counter(round(winter_dataset['Temperature (C)']))}")
print(f"Humi-count: {collections.Counter(round(winter_dataset['Relative Humidity (%)']))}")
print(f"Pressure-count: {collections.Counter(round(winter_dataset['Pressure (hPa)']))}")

mean-temperature: 26.816446148563603
mean-humidity: 62.7195135630079
mean-pressure: 997.1622811033535
Temp-countCounter({26.0: 1187, 27.0: 1163, 25.0: 1136, 28.0: 877, 24.0: 857, 29.0: 641, 23.0: 631, 30.0: 620, 31.0: 496, 32.0: 365, 22.0: 338, 33.0: 167, 21.0: 116, 34.0: 88, 20.0: 39, 35.0: 12, 36.0: 3, 19.0: 1})
Humi-count: Counter({64.0: 212, 65.0: 202, 60.0: 193, 66.0: 193, 88.0: 188, 69.0: 184, 61.0: 183, 67.0: 182, 63.0: 181, 70.0: 176, 89.0: 175, 68.0: 175, 59.0: 174, 62.0: 174, 71.0: 170, 55.0: 169, 58.0: 166, 56.0: 161, 57.0: 160, 54.0: 156, 76.0: 154, 73.0: 154, 72.0: 154, 77.0: 153, 79.0: 149, 80.0: 147, 74.0: 147, 87.0: 144, 86.0: 144, 75.0: 144, 81.0: 139, 53.0: 139, 90.0: 137, 83.0: 137, 85.0: 135, 78.0: 132, 50.0: 129, 82.0: 127, 41.0: 126, 52.0: 123, 46.0: 122, 42.0: 122, 45.0: 121, 84.0: 120, 40.0: 120, 51.0: 120, 48.0: 118, 47.0: 117, 49.0: 109, 44.0: 108, 43.0: 108, 39.0: 105, 38.0: 102, 37.0: 97, 36.0: 96, 34.0: 94, 35.0: 87, 91.0: 82, 33.0: 65, 32.0: 63, 31.0: 50, 

In [689]:
# plotting Temperature graph
dummy = winter_dataset.copy()
px.scatter(dummy, x='time', y='Temperature (C)')

In [690]:
# plotting Pressure graph
dummy = winter_dataset.copy()
px.scatter(dummy, x='time', y='Pressure (hPa)')

In [691]:
# Scaling data
dTemp = dummy[['TIME', "Temperature (C)"]]
dTemp1 = dTemp.copy()
dTemp[['TIME', "Temperature (C)"]] = StandardScaler().fit_transform(dTemp.values)
dHumi = dummy[['TIME', "Relative Humidity (%)"]]
dHumi1 = dHumi.copy()
dHumi[['TIME', "Realtive Humidity (%)"]] = StandardScaler().fit_transform(dHumi.values)
dPres = dummy[['TIME', "Pressure (hPa)"]]
dPres1 = dPres.copy()
dPres[['TIME', "Pressure (hPa)"]] = StandardScaler().fit_transform(dPres.values)

In [692]:
# getting epsilon value for temperature
df = dummy
Tmins = len(df)//300
nn = NearestNeighbors(n_neighbors=Tmins+1)
nn.fit(dTemp[['TIME',"Temperature (C)"]])
distances, neighbors = nn.kneighbors(dTemp[['TIME',"Temperature (C)"]])

# sort the distances
distances = np.sort(distances[:, Tmins], axis = 0)

distances_df = pd.DataFrame({"distances": distances,
                             "index": list(range(0,len(distances)))})

def calc_distance(x1, y1, a, b, c):
  d = abs((a * x1 + b * y1 + c)) / (math.sqrt(a * a + b * b))
  return d

def find_epsTemp():
  # (y1 – y2)x + (x2 – x1)y + (x1y2 – x2y1) = 0
  a = distances[0] - distances[-1]  #y
  b = distances_df.index[-1] - distances_df.index[0]    #x
  c1 = distances_df.index[0] * distances[-1]
  c2 = distances_df.index[-1] * distances[0]
  c = c1 - c2

  distance_of_points_from_line = []
  for k in range(len(distances_df)):
    distance_of_points_from_line.append(
        calc_distance(distances_df.index[k], distances[k], a, b, c))
    
  dist = pd.Series(distance_of_points_from_line)
  index_max = dist.idxmax()
  return distances[index_max]

In [693]:
# getting epsilon value for pressure
df = dummy
Pmins = len(df)//300
nn = NearestNeighbors(n_neighbors=Pmins+1)
nn.fit(dPres[['TIME',"Pressure (hPa)"]])
distances, neighbors = nn.kneighbors(dPres[['TIME',"Pressure (hPa)"]])

# sort the distances
distances = np.sort(distances[:, Pmins], axis = 0)

distances_df = pd.DataFrame({"distances": distances, "index": list(range(0,len(distances)))})

def calc_distance(x1, y1, a, b, c): 
  d = abs((a * x1 + b * y1 + c)) / (math.sqrt(a * a + b * b))
  return d

def find_epsPres():
  # (y1 – y2)x + (x2 – x1)y + (x1y2 – x2y1) = 0
  a = distances[0] - distances[-1]  #y
  b = distances_df.index[-1] - distances_df.index[0]    #x
  c1 = distances_df.index[0] * distances[-1]
  c2 = distances_df.index[-1] * distances[0]
  c = c1 - c2
  distance_of_points_from_line = []
  for k in range(len(distances_df)):
    distance_of_points_from_line.append(calc_distance(distances_df.index[k], distances[k], a, b, c))
  dist = pd.Series(distance_of_points_from_line)
  index_max = dist.idxmax()
  return distances[index_max]

In [694]:
# # getting epsilon value for Humidity
# df = dummy
# Hmins = len(df)//500
# nn = NearestNeighbors(n_neighbors=Hmins+1)
# nn.fit(dHumi[['TIME',"Realtive Humidity (%)"]])
# distances, neighbors = nn.kneighbors(dHumi[['TIME',"Realtive Humidity (%)"]])

# # sort the distances
# distances = np.sort(distances[:, Hmins], axis = 0)

# distances_df = pd.DataFrame({"distances": distances,
#                              "index": list(range(0,len(distances)))})


# def calc_distance(x1, y1, a, b, c):
#   d = abs((a * x1 + b * y1 + c)) / (math.sqrt(a * a + b * b))
#   return d

# def find_epsHumi():
#   # (y1 – y2)x + (x2 – x1)y + (x1y2 – x2y1) = 0
#   a = distances[0] - distances[-1]  #y
#   b = distances_df.index[-1] - distances_df.index[0]    #x
#   c1 = distances_df.index[0] * distances[-1]
#   c2 = distances_df.index[-1] * distances[0]
#   c = c1 - c2

#   distance_of_points_from_line = []
#   for k in range(len(distances_df)):
#     distance_of_points_from_line.append(
#         calc_distance(distances_df.index[k], distances[k], a, b, c))
    
#   dist = pd.Series(distance_of_points_from_line)
#   index_max = dist.idxmax()
#   return distances[index_max] 

In [695]:

# # For Humidity
# nn.fit(dHumi[['TIME',"Relative Humidity (%)"]])
# distance2, neighbors = nn.kneighbors(dHumi[['TIME',"Relative Humidity (%)"]])

# # sort the distances
# distance2= np.sort(distance2[:, Hmins], axis = 0)

# #plot the distances
# distance2_df = pd.DataFrame({"distances-Humi": distance2,
#                               "index": list(range(0,len(distance2)))})

# plt2 = (ggplot(distance2_df, aes(x = "index", y = "distances-Humi")) +
#  geom_line(color = "white", size = 2) + theme_minimal() +
#  labs(title = "Elbow Method for Choosing eps") +
#  theme(panel_grid_minor = element_blank(),
#       rect = element_rect(fill = "#202124ff"),
#       axis_text = element_text(color = "white"),
#       axis_title = element_text(color = "white"),
#       plot_title = element_text(color = "white"),
#       panel_border = element_line(color = "darkgray"),
#       plot_background = element_rect(fill = "#202124ff")
#       ))
# ggsave(plot=plt2, filename='elbow.png', dpi=300)

# plt2      #eps=1

In [696]:
# applying dbscan for Temperature
db1 = DBSCAN(eps = find_epsTemp(), min_samples = Tmins).fit(dTemp)
labsList1 = ["Noise"]
labsList1 = labsList1 + ["Cluster " + str(i) for i in range(1,len(set(db1.labels_)))]
dTemp["assignments"] = db1.labels_
dTemp1["assignments"] = db1.labels_
color1 = dTemp["assignments"].astype(int)
n_noise_1 = list(db1.labels_).count(-1)
print("Estimated number of noise points for Temperature: %d" % n_noise_1)

# applying dbscan for Pressure
db3 = DBSCAN(eps = find_epsPres(), min_samples = Pmins).fit(dPres)
labsList3 = ["Noise"]
labsList3 = labsList3 + ["Cluster " + str(i) for i in range(1,len(set(db3.labels_)))]
dPres["assignments"] = db3.labels_
dPres1["assignments"] = db3.labels_
color3 = dPres["assignments"].astype(int)
n_noise_3 = list(db3.labels_).count(-1)
print("Estimated number of noise points for Pressure: %d" % n_noise_3)



Estimated number of noise points for Temperature: 237
Estimated number of noise points for Pressure: 308


In [697]:
# # applying dbscan for Humidity
# db2 = DBSCAN(eps = 1, min_samples = Hmins).fit(dHumi)
# labsList2 = ["Noise"]
# labsList2 = labsList2 + ["Cluster " + str(i) for i in range(1,len(set(db2.labels_)))]
# dHumi["assignments"] = db2.labels_
# color2 = dHumi["assignments"].astype(int)
# n_noise_2 = list(db2.labels_).count(-1)
# print("Estimated number of noise points: %d" % n_noise_2)

In [698]:
# # applying dbscan for Pressure
# db3 = DBSCAN(eps = find_epsPres(), min_samples = Pmins).fit(dPres)
# labsList3 = ["Noise"]
# labsList3 = labsList3 + ["Cluster " + str(i) for i in range(1,len(set(db3.labels_)))]
# dPres["assignments"] = db3.labels_
# dPres1["assignments"] = db3.labels_
# color3 = dPres["assignments"].astype(int)
# n_noise_3 = list(db3.labels_).count(-1)
# print("Estimated number of noise points: %d" % n_noise_3)

In [699]:
# plotting graph with anomalies for Temperature
dTemp1['time'] = dummy['time']
px.scatter(dTemp1, x='time', y='Temperature (C)', color=color1)

In [700]:
# # plotting graph with anomalies for Humudity
# dHumi['time'] = dummy['time']
# px.scatter(dHumi, x='time', y='Relative Humidity (%)', color = color2)

In [701]:
# plotting graph with anomalies for Pressure
dPres1['time'] = dummy['time']
px.scatter(dPres1, x='time', y='Pressure (hPa)', color = color3)

In [702]:
#getting anomaly data
Pres_anomaly = dPres1[dPres1['assignments'] == -1]
Pres_anomaly = Pres_anomaly[['time','Pressure (hPa)',]]
Temp_anomaly = dTemp1[dTemp1['assignments'] == -1]
Temp_anomaly = Temp_anomaly[['time','Temperature (C)',]]

In [703]:
AAA = pd.concat([Temp_anomaly, Pres_anomaly], axis=0, ignore_index=True)
AAA = AAA.sort_values(by=['time'])
AAA.reset_index(drop=True, inplace=True)
AAA.replace(np.nan, 0.000, inplace=True)

for i in range(543):
  if AAA['time'][i] == AAA['time'][i+1]:
    AAA['Temperature (C)'][i] = AAA['Temperature (C)'][i] + AAA['Temperature (C)'][i+1]
    AAA['Pressure (hPa)'][i] = AAA['Pressure (hPa)'][i] + AAA['Pressure (hPa)'][i+1]
    AAA['Pressure (hPa)'][i+1] = math.nan
    AAA['Temperature (C)'][i+1] = math.nan

AAA = AAA[AAA['Temperature (C)'].notna()]
AAA.replace(0.000, '-', inplace=True)
AAA.head(50)

In [706]:
# # anomalous data
# df1 = winter.copy()
# anomaly = d1[d1.assignments == -1]
# anomalies_index = list(anomaly.index)
# temp_list = []
# time_list = []
# humidity_list = []
# pressure_list = []
# # ["Hour","Temperature (C)","Humidity", "Pressure (millibars)"]
# for index in anomalies_index:
#   temp = df1.loc[index]['Temperature (C)']
#   temp_list.append(temp)
#   time = df1.loc[index]['time']
#   time_list.append(time)
#   humidity = df1.loc[index]['Relative Humidity (%)']
#   humidity_list.append(humidity)
#   pressure = df1.loc[index]['Pressure (hPa)']
#   pressure_list.append(pressure)
# dic = {
#     'Time': time_list,
#     'Temperature': temp_list,
#     'Humidity': humidity_list,
#     'Pressure': pressure_list
# }

# anomaly_df = pd.DataFrame(dic)

# # determining the name of the file
# file_name = 'Anomaly_df.csv'
  
# # saving the excel
# anomaly_df.to_csv(file_name)

# anomaly_df


In [707]:
# winter_temp_anomaly = anomaly_df[['Time','Temperature']]
# winter_temp_anomaly1 = winter_temp_anomaly[winter_temp_anomaly['Time'].dt.year == 2017]
# winter_temp_anomaly1 = winter_temp_anomaly1[winter_temp_anomaly['Time'].dt.month.isin([2, 1])]
# winter_temp = winter[['time',"Temperature (C)"]]
# winter_temp1 = winter_temp[winter_temp['time'].dt.year == 2017]
# winter_temp1 = winter_temp1[winter_temp['time'].dt.month.isin([2, 1])]

# winter_pressure_anomaly = anomaly_df[['Time','Pressure']]
# winter_pressure_anomaly1 = winter_pressure_anomaly[winter_pressure_anomaly['Time'].dt.year == 2017]
# winter_pressure_anomaly1 =winter_pressure_anomaly1[winter_pressure_anomaly['Time'].dt.month.isin([2, 1])]
# winter_pressure = winter[['time',"Pressure (hPa)"]]
# winter_pressure1 = winter_pressure[winter_pressure['time'].dt.year == 2017]
# winter_pressure1 = winter_pressure1[winter_pressure['time'].dt.month.isin([2, 1])]

# winter_humi_anomaly = anomaly_df[['Time','Humidity']]
# winter_humi_anomaly1 = winter_humi_anomaly[winter_humi_anomaly['Time'].dt.year == 2017]
# winter_humi_anomaly1 = winter_humi_anomaly1[winter_humi_anomaly['Time'].dt.month.isin([2, 1])]
# winter_humi = winter[['time',"Relative Humidity (%)"]]
# winter_humi1 = winter_humi[winter_humi['time'].dt.year == 2017]
# winter_humi1 = winter_humi1[winter_humi['time'].dt.month.isin([2, 1])]


# from plotly.subplots import make_subplots
# import plotly.graph_objects as go

# fig = make_subplots(rows=3, cols=1)

# #temperature
# fig.add_trace(go.Scatter(x=winter_temp1['time'], y=winter_temp1['Temperature (C)'],
#                     marker=dict(size=3),
#                     mode='markers',
#                     name='Temp-lines'),row=1, col=1)
# fig.add_trace(go.Scatter(x=winter_temp_anomaly1['Time'], y= winter_temp_anomaly1['Temperature'],
#                     marker=dict(size=5),
#                     mode='markers',
#                     name='anomaly'),row=1, col=1)

# #Pressure
# fig.add_trace(go.Scatter(x=winter_pressure1['time'], y=winter_pressure1['Pressure (hPa)'],
#                     marker=dict(size=3),
#                     mode='markers',
#                     name='Pressure-lines'),row=2, col=1)
# fig.add_trace(go.Scatter(x=winter_pressure_anomaly1['Time'], y= winter_pressure_anomaly1['Pressure'],
#                     marker=dict(size=5),
#                     mode='markers',
#                     name='anomaly'),row=2, col=1)

# #humidity
# fig.add_trace(go.Scatter(x=winter_humi1['time'], y=winter_humi1['Relative Humidity (%)'],
#                     marker=dict(size=3),
#                     mode='markers',
#                     name='Humidity-lines'),row=3, col=1)
# fig.add_trace(go.Scatter(x=winter_humi_anomaly1['Time'], y= winter_humi_anomaly1['Humidity'],
#                     marker=dict(size=5),
#                     mode='markers',
#                     name='anomaly'),row=3, col=1)

# fig.update_layout(height=1200, title_text="Anomalies in Temperature, Pressure, Humidity")
# # edit axis labels
# fig['layout']['xaxis']['title']='Time'
# fig['layout']['xaxis2']['title']='Time'
# fig['layout']['xaxis3']['title']='Time'
# fig['layout']['yaxis']['title']='Temperature'
# fig['layout']['yaxis2']['title']='Pressure'
# fig['layout']['yaxis3']['title']='Humidity'

# fig.show()


